In [217]:
import re
import os
import yaml
import json
import pandas as pd
from google.cloud import bigquery
import google.api_core.exceptions as google_exceptions

In [218]:
config  = yaml.safe_load(open('config.yaml', 'r'))

In [219]:
query_dir = 'queries'

In [220]:
queries = {}
for query_file in (os.listdir(query_dir)):
    with open(os.path.join(query_dir, query_file), 'r') as query:
        queries[query_file] = query.read()

In [221]:
query_client = bigquery.Client()

In [222]:
def exists_table(table_reference, client):
    try:
        client.get_table(table_reference)
        return True
    except google_exceptions.NotFound:
        return False

In [223]:
if exists_table(f'{config["project_id"]}.{config["dataset"]}.expanded_sub_properties', query_client):
    request = queries['all_new_buildings_that_have_not_been_expanded.sql']
else:
    request = queries['all_new_buildings.sql']

In [224]:
real_estate_raw = query_client.query(request).to_dataframe()

In [225]:
real_estate_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 5 columns):
ad_id             129 non-null int64
new_building      129 non-null bool
property_table    129 non-null object
property_attrs    129 non-null object
address           129 non-null object
dtypes: bool(1), int64(1), object(3)
memory usage: 4.2+ KB


In [226]:
real_estate_raw.set_index(keys='ad_id', drop=True, inplace=True)

In [227]:
real_estate_raw.head()

,new_building,property_table,property_attrs,address
ad_id,,,,
85287718,True,"{""Bolig"": [""120"", ""143"", ""183"", ""187"", ""204"", ...","{""Areal"": ""43\u00a0-\u00a090 m\u00b2"", ""Sovero...","Innspurten 6, 0663 Oslo"
91411206,True,"{""Bolig"": [""104"", ""1004"", ""2001"", ""2003"", ""300...","{""Fellesgjeld"": ""2\u00a0790\u00a0000 \u2013 5\...","Lillebergtunet borettslag, Hus A, trinn 1, 066..."
93367028,True,"{""Bolig"": [""302"", ""305"", ""307"", ""402"", ""407"", ...","{""Fellesgjeld"": ""1\u00a0900\u00a0000 \u2013 6\...","Frysjaveien 42 - trinn 1, 0884 Oslo"
95247107,True,"{""Bolig"": [""B4009"", ""B4010"", ""B4013"", ""B5012"",...","{""Fellesgjeld"": ""2\u00a0430\u00a0000 \u2013 5\...","Lillebergtunet borettlslag, Hus B, trinn 2, Ma..."
96760455,True,"{""Bolig"": [""A303"", ""B401"", ""C201"", ""C202"", ""C3...","{""Omkostninger"": ""1\u00a0064 kr"", ""Totalpris"":...","Røa Torg - Vækerøveien 195, 0751 Oslo"


In [228]:
for col in ['property_table', 'property_attrs']:
    real_estate_raw[col] = [json.loads(cell) for cell in real_estate_raw[col]]

In [229]:
def expanand_property_tables(new_buildings):
    cols = new_buildings.iloc[0]['property_table'].keys()
    col_map = {'Bolig': 'apt_id',
               'Soverom': 'num_bedrooms',
               'Etg': 'floor',
               'P-rom': 'primary_size',
               'BRA': 'total_size',
               'Totalpris': 'price',
               'Pris': 'price'}
    property_tables = pd.DataFrame(columns=cols)
    for code, row in new_buildings['property_table'].iteritems():
        all_properties = [arr for arr in row.values()]
        to_append = dict(zip(cols, all_properties))
        frame = pd.DataFrame(to_append, columns=cols)
        frame['ad_id'] = code
        property_tables = property_tables.append(frame, ignore_index=True, sort=False)
    property_tables.rename(columns=col_map, inplace=True)
    return property_tables

In [230]:
sub_properties = expanand_property_tables(real_estate_raw[(real_estate_raw.new_building == True) & (real_estate_raw.property_table)])

In [231]:
sub_properties.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,120,2,1.,48 m²,59 m²,4 150 000 kr,85287718.0
1,143,2,2.,56 m²,61 m²,4 350 000 kr,85287718.0
2,183,2,5.,51 m²,59 m²,4 350 000 kr,85287718.0
3,187,2,5.,48 m²,59 m²,4 350 000 kr,85287718.0
4,204,3,6.,86 m²,92 m²,8 495 000 kr,85287718.0


In [232]:
def get_int_from_str(string):
    try:
        concat_string = ''.join(re.findall(r'[\d.]', string))
        if concat_string != '':
            return float(concat_string)
    except Exception as e:
        print(e, string)
        pass

In [233]:
sub_properties_parsed = sub_properties.copy()
for col in ['primary_size', 'floor', 'price', 'num_bedrooms', 'total_size']:
    sub_properties_parsed[col] = [get_int_from_str(s) for s in sub_properties_parsed[col]]

expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None


In [234]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573 entries, 0 to 1572
Data columns (total 7 columns):
apt_id          1573 non-null object
num_bedrooms    1571 non-null float64
floor           1350 non-null float64
primary_size    1238 non-null float64
total_size      1570 non-null float64
price           1563 non-null float64
ad_id           1573 non-null float64
dtypes: float64(6), object(1)
memory usage: 86.1+ KB


In [235]:
sub_properties_parsed.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,120,2.0,1.0,48.0,59.0,4150000.0,85287718.0
1,143,2.0,2.0,56.0,61.0,4350000.0,85287718.0
2,183,2.0,5.0,51.0,59.0,4350000.0,85287718.0
3,187,2.0,5.0,48.0,59.0,4350000.0,85287718.0
4,204,3.0,6.0,86.0,92.0,8495000.0,85287718.0


In [236]:
sub_properties_parsed.drop_duplicates(inplace=True)

In [237]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092 entries, 0 to 1542
Data columns (total 7 columns):
apt_id          1092 non-null object
num_bedrooms    1090 non-null float64
floor           914 non-null float64
primary_size    913 non-null float64
total_size      1091 non-null float64
price           1084 non-null float64
ad_id           1092 non-null float64
dtypes: float64(6), object(1)
memory usage: 68.2+ KB


In [238]:
sub_properties_parsed.to_gbq(destination_table='housing_data.expanded_sub_properties',
                             project_id='hde-test-clean',
                             if_exists='append')